In [3]:
import tensorflow as tf  
import numpy as np       
import pandas as pd       
from matplotlib import pyplot as plt   
import seaborn as sns  
import time          
from tensorflow.keras.callbacks import History, ReduceLROnPlateau, TensorBoard       
from tensorflow.keras import metrics          
from sklearn.metrics import confusion_matrix          
import pickle
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, multiply, add, Dense, Conv2D, GlobalMaxPooling2D, Activation
from tensorflow.keras import backend as K
import platform
import os
from tensorflow.keras.layers import GlobalAveragePooling2D, Multiply, Dense
from keras.layers import GlobalAveragePooling2D, Dense, Multiply
import platform
import os

In [4]:
# Display Settings
pd.options.display.max_columns = None
pd.options.display.max_rows = None
sns.set()

In [5]:
# System Information Print the Python version
print("Python version: ", platform.python_version())
print("TensorFlow version: ", tf.__version__)
print("Current working directory: ", os.getcwd())
!uname -a
!nvidia-smi

Python version:  3.11.7
TensorFlow version:  2.15.0
Current working directory:  /root/.jupyter/张彤/消融实验
Linux w3q2ulc9.vm 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux
Sun Oct 27 11:09:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:09:00.0 Off |                  Off |
|  0%   24C    P8              11W

In [6]:
# Hyperparameter Settings
EPOCHS = 50
IMAGE_SIZE = (224, 224)
IMAGE_PATH = "../data"
LEARNING_RATE = 1e-4
BATCH_SIZE = 64

In [7]:
# Load Dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="training",    
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

vaild_ds = tf.keras.utils.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 14080 files belonging to 100 classes.
Using 11264 files for training.


2024-10-27 11:09:25.865234: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-27 11:09:25.865724: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-27 11:09:26.154632: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Found 14080 files belonging to 100 classes.
Using 2816 files for validation.


In [8]:
# Image Normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

def normalize_image(image):
    return (image - mean_tensor) / std_tensor


In [9]:
# Data Augmentation
train_image_augment = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1 / 255.0),
    tf.keras.layers.RandomRotation(factor=0.2),
    tf.keras.layers.RandomFlip(),
])

valid_image_augment = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1 / 255.0),
])

In [10]:
# Input Processing
def process_train_input(images, labels):
    return train_image_augment(images), labels

def process_valid_input(images, labels):
    return valid_image_augment(images), labels


In [11]:
# Data Preprocessing
def convert_types_and_encode(x, y):
    y = tf.cast(y, tf.int32)  
    y_one_hot = tf.one_hot(y, 100)  
    return x, y_one_hot

In [12]:
# Apply Data Preprocessing
train_ds = train_ds.map(convert_types_and_encode)
train_ds = train_ds.map(process_train_input, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

vaild_ds = vaild_ds.map(convert_types_and_encode)
vaild_ds = vaild_ds.map(process_valid_input, num_parallel_calls=tf.data.AUTOTUNE)
vaild_ds = vaild_ds.prefetch(tf.data.AUTOTUNE)

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Multiply, Dense
import tensorflow.keras.backend as K

# Define the modified ECA layer, which now supports partial channel selection
def eca_layer(input_tensor, k_size=3, partial_ratio=0.75):
    """Efficient Channel Attention (ECA) for a portion of the channels.
    
    Args:
        input_tensor: The input feature map (batch_size, height, width, channels).
        k_size: Kernel size for the 1D convolution used in ECA.
        partial_ratio: Ratio of channels to apply ECA on. In this case, 0.75 means applying ECA on 75% of the channels.
    
    Returns:
        The output tensor after applying ECA on the selected channels.
    """
    channels = input_tensor.shape[-1]  # Get the total number of channels
    partial_channels = int(channels * partial_ratio)  # Calculate the number of channels to apply ECA

    # Split the input tensor into two parts: the first part to apply ECA and the second part to leave unchanged
    eca_part = input_tensor[:, :, :, :partial_channels]
    non_eca_part = input_tensor[:, :, :, partial_channels:]
    
    # Apply global average pooling to the selected channels
    avg_pool = tf.reduce_mean(eca_part, axis=[1, 2], keepdims=True)  # (batch_size, 1, 1, partial_channels)
    
    # Apply 1D convolution (as Conv2D with (k_size, 1) kernel) to capture cross-channel dependencies
    conv = Conv2D(filters=partial_channels, kernel_size=(k_size, 1), padding='same', activation='sigmoid')(avg_pool)
    
    # Multiply the attention map with the input feature maps (only for the selected channels)
    weighted_eca_part = Multiply()([eca_part, conv])  # (batch_size, height, width, partial_channels)
    
    # Concatenate the processed ECA part with the unchanged part
    output = tf.concat([weighted_eca_part, non_eca_part], axis=-1)

    return output

# Create MobileNet base model (or any other backbone)
base_model = tf.keras.applications.MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Select the low-level layers to apply ECA
# For low-level features, we can apply ECA to the first couple of layers like 'conv1_relu' and 'conv_dw_1_relu'
selected_layers = ['conv1_relu', 'conv_dw_1_relu']  # Example low-level layers, you can modify based on your needs

# Apply ECA only to the selected layers (low-level features)
x = base_model.input
for layer in base_model.layers:
    x = layer(x)
    if layer.name in selected_layers:
        x = eca_layer(x, k_size=5, partial_ratio=0.75)  # Apply ECA to the selected low-level layers with 75% channels

# Add global average pooling
x = GlobalAveragePooling2D()(x)

# Add fully connected layers
x = Dense(1024, activation='relu')(x)
outputs = Dense(100, activation='softmax')(x)  # Adjust the number of classes as needed

# Create the final model
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [14]:
# Compile Model
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss_fn = tf.keras.losses.CategoricalCrossentropy()
metrics = [
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall(),
]

model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

In [15]:
# Train Model
log_dir = "../Running result/substratum/substratum"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=2, min_lr=1e-8)

history = model.fit(train_ds, epochs=EPOCHS, validation_data=vaild_ds, callbacks=[lr_scheduler, tensorboard_callback])

Epoch 1/50


2024-10-27 11:09:42.182104: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-10-27 11:09:42.471795: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-27 11:09:43.674054: I external/local_xla/xla/service/service.cc:168] XLA service 0x7ff1d0a9df60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-27 11:09:43.674138: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-10-27 11:09:43.674154: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-10-27 11:09:43.706105: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1729998583.990257    1619 device_comp

176/176 [==============================] - 38s 94ms/step - loss: 2.6143 - categorical_accuracy: 0.4259 - precision: 0.8905 - recall: 0.1768 - val_loss: 2.8704 - val_categorical_accuracy: 0.2962 - val_precision: 0.6391 - val_recall: 0.1126 - lr: 1.0000e-04
Epoch 2/50
176/176 [==============================] - 15s 84ms/step - loss: 1.0027 - categorical_accuracy: 0.7391 - precision: 0.9043 - recall: 0.5763 - val_loss: 1.0624 - val_categorical_accuracy: 0.7138 - val_precision: 0.8715 - val_recall: 0.5589 - lr: 1.0000e-04
Epoch 3/50
176/176 [==============================] - 15s 84ms/step - loss: 0.6247 - categorical_accuracy: 0.8307 - precision: 0.9240 - recall: 0.7358 - val_loss: 0.7480 - val_categorical_accuracy: 0.7926 - val_precision: 0.8856 - val_recall: 0.7038 - lr: 1.0000e-04
Epoch 4/50
176/176 [==============================] - 15s 84ms/step - loss: 0.4453 - categorical_accuracy: 0.8797 - precision: 0.9428 - recall: 0.8169 - val_loss: 0.5830 - val_categorical_accuracy: 0.8342 - val

In [16]:
import pandas as pd  # Import the Pandas library
import os

# Define save paths
model_dir = "../Running result/substratum/substratum"
excel_file_path = os.path.join(model_dir, "substratum.xlsx")  # Path to save the Excel file
model_file_path = os.path.join(model_dir, "substratum.h5")  # Path to save the model

# Check if the save directory exists, and create it if it does not
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save training history to an Excel file
history_df = pd.DataFrame(history.history)  
history_df.to_excel(excel_file_path, index=False)
print(f"Training history saved to: {excel_file_path}")

# Save the model to the specified path
model.save(model_file_path)
print(f"Model saved to: {model_file_path}")


Training history saved to: ../Running result/substratum/substratum/substratum.xlsx


/root/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to: ../Running result/substratum/substratum/substratum.h5


In [1]:
import os
import time
import numpy as np
import tensorflow as tf
import random
from PIL import Image
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Multiply, Dense, Input

# 强制同步 GPU 操作，确保计时准确
tf.config.experimental.set_synchronous_execution(True)

# 定义 ECA 层 
def eca_layer(input_tensor, k_size=3, partial_ratio=0.75):
    channels = input_tensor.shape[-1]
    partial_channels = int(channels * partial_ratio)

    eca_part = input_tensor[:, :, :, :partial_channels]  # 仅部分通道参与ECA
    non_eca_part = input_tensor[:, :, :, partial_channels:]  # 剩余通道不参与ECA
    
    avg_pool = tf.reduce_mean(eca_part, axis=[1, 2], keepdims=True)  # 全局平均池化
    conv = Conv2D(filters=partial_channels, kernel_size=(k_size, 1), padding='same', activation='sigmoid')(avg_pool) 
    
    weighted_eca_part = Multiply()([eca_part, conv])  # 加权操作
    
    output = tf.concat([weighted_eca_part, non_eca_part], axis=-1)  # 合并参与ECA和不参与的通道
    return output

# 创建模型并应用 ECA 层 
def create_model():
    inputs = Input(shape=(224, 224, 3))

    # 使用 MobileNet 作为基础模型，不包括顶层
    base_model = tf.keras.applications.MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet', input_tensor=inputs)
    
    # 选择低层特征层应用 ECA
    selected_layers = ['conv1_relu', 'conv_dw_1_relu']  # 这些是MobileNet的低层特征层
    
    x = inputs
    for layer in base_model.layers:
        x = layer(x)
        if layer.name in selected_layers:
            x = eca_layer(x, k_size=3, partial_ratio=0.75)  
    
    x = GlobalAveragePooling2D()(x)  
    x = Dense(1024, activation='relu')(x)
    outputs = Dense(100, activation='softmax')(x)  
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# 获取图片文件
def get_image_files(dataset_dir):
    image_files = []
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                image_files.append(os.path.join(root, file))
    return image_files

# 处理图片 
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

#  GPU 同步与缓存清理 
def gpu_synchronize_and_clear():
    # 强制 GPU 同步，确保所有任务完成后再计时
    tf.config.experimental.set_synchronous_execution(True)
    # 清理 GPU 缓存
    tf.keras.backend.clear_session()

# 数据预加载 
def preload_all_images_to_memory(image_files):
    """预加载所有图片到内存中，避免推理时频繁进行磁盘读取。"""
    preloaded_images = {}
    for image_file in image_files:
        img_array = load_and_preprocess_image(image_file)
        preloaded_images[image_file] = img_array
    return preloaded_images

#  推理函数 
def inference_with_separate_timing(model, images, num_images):
    gpu_synchronize_and_clear()  # 清理缓存并同步 GPU

    # 执行推理并计时
    t1 = time.time()
    model.predict(images)
    gpu_synchronize_and_clear()  # 再次同步 GPU

    t2 = time.time()

    inference_time = t2 - t1  # 记录推理时间
    return inference_time

# 单张图片推理 -
def single_image_inference(model, image_files):
    random_image_file = random.choice(image_files)
    
    # 记录图片加载和预处理时间
    image_data = load_and_preprocess_image(random_image_file)
    
    # 执行推理并记录推理时间
    inference_time = inference_with_separate_timing(model, image_data, 1)
    return inference_time, random_image_file

# 批量图片推理 
def batch_image_inference_and_async_loading(model, preloaded_images, image_files, batch_size):
    t1 = time.time()
    for img in random.sample(image_files, batch_size):
        img = preloaded_images[img]
        model.predict(img)
    t2 = time.time()
    inference_time = t2 - t1
    average_time_per_image = inference_time / batch_size
    return inference_time, average_time_per_image

# 主程序入口 
if __name__ == "__main__":
    dataset_dir = "../data"  # 设置数据集目录
    model = create_model()

    # 模型预热
    model(np.zeros((1, 224, 224, 3)))

    # 获取图片文件
    image_files = get_image_files(dataset_dir)

    # 预加载所有图片到内存
    preloaded_images = preload_all_images_to_memory(image_files)

    # 只计算1张、500张和1000张的推理时间
    for batch_size in [1, 500, 1000]:
        if batch_size == 1:
            inference_time, image_used = single_image_inference(model, image_files)
            print(f"Single image inference time: {inference_time * 1000:.4f} ms")
        else:
            inference_time, avg_time_per_image = batch_image_inference_and_async_loading(model, preloaded_images, image_files, batch_size)
            print(f"Batch of {batch_size} images inference time: {inference_time * 1000:.4f} ms")
            print(f"Average time per image in batch of {batch_size}: {avg_time_per_image * 1000:.4f} ms")


2024-10-27 11:43:03.758262: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-27 11:43:03.758625: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-27 11:43:03.818627: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


2024-10-27 11:43:05.995175: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-10-27 11:43:06.108341: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-27 11:43:06.579327: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 0s 429ms/step
Single image inference time: 497.5176 ms
1/1 [==============================] - 0s 34ms/step
Batch of 500 images inference time: 51071.5766 ms
Average time per image in batch of 500: 102.1432 ms
1/1 [==============================] - 0s 32ms/step
Batch of 1000 images inference time: 101930.3024 ms
Average time per image in batch of 1000: 101.9303 ms
